In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

from scipy.optimize import minimize

# Choosing target companies

In [2]:
# TODO choose 10 / 50 every week
# TODO add target_short
target_long = pd.DataFrame(['ASML', 'GOOG', 'ACN', 'AAPL', 'SONY', 'INTU', 'V', 'MA', 'BX', 'TTE'],
                           columns=['Ticker'])

# Calculating portfolio weights

## Getting 2-month history

In [3]:
def get_history(target_df: pd.DataFrame) -> dict:
    history = pd.DataFrame(columns=list(target_long['Ticker']))
    for ticker in target_df['Ticker']:
        history[ticker] = yf.Ticker(ticker).history(interval='5m')['Close']
    return history

In [4]:
history_long = get_history(target_long)


## Getting 2-month compound returns

In [5]:
def get_returns(history_df: pd.DataFrame) -> np.ndarray:
    n = len(history_df.columns)
    returns = np.zeros(n, dtype=float)
    for i in range(n):
        # TODO think about appropriate return calculation
        helper = history_df.iloc[:, i] / history_df.iloc[:, i].shift(1)
        helper = helper.iloc[1:]
        returns[i] = np.prod(helper) - 1
    return returns

In [6]:
returns_long = get_returns(history_long)
print(f'Top companies\' returns: {returns_long}')

Top companies' returns: [ 0.08943322 -0.03332119  0.01574109  0.0321775   0.05649513 -0.02432639
  0.04853496 -0.01120826 -0.0429326   0.0073824 ]


## Getting weights

In [7]:
n = len(target_long['Ticker'])
weights_long = np.ones(n)


def sharpe_long(weights):
    return -np.dot(returns_long, weights) / np.sqrt(weights @ history_long.cov().to_numpy() @ weights)


bounds = [(0, None)] * n
constraints = ({'type': 'eq', 'fun': lambda weights: weights.sum() - 1.0})
optimal_weights_long = minimize(sharpe_long, weights_long, method='SLSQP',
                                constraints=constraints,
                                bounds=bounds
                                ).x

print(f'Optimal weights: {optimal_weights_long}')
print(f'Optimal Sharpe ratio: {-sharpe_long(optimal_weights_long)}')

Optimal weights: [0.         0.         0.         0.         0.76477022 0.
 0.         0.         0.         0.23522981]
Optimal Sharpe ratio: 0.02893133412227156
